### PEGASUS
+ 使用 cnn dailymail 通过 PEGASUS 进行模型效果评估

In [1]:
# 代理必须关闭
# hide_output
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail",version="3.0.0")
print(f"Features: {dataset['train'].column_names}")

Using custom data configuration default
Found cached dataset cnn_dailymail (C:/Users/Administrator/.cache/huggingface/datasets/cnn_dailymail/default/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

Features: ['article', 'highlights', 'id']


In [2]:
sample = dataset["train"][1]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["article"])}):
""")
print(sample["article"][:500])
print(f'\nSummary (length: {len(sample["highlights"])}):')
print(sample["highlights"])



Article (excerpt of 500 characters, total length: 4051):

Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

Summary (length: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


In [9]:
sample_text = dataset["train"][1]["article"][:2000]
sample_text = '在海下平静的航程中，沃纳睡着了。不知过了多长时间，他被霍普金斯推醒，他看看小屏幕上的海图和航迹，发现航程已走了三分之二，似乎没有什么异常。霍普金斯让他注意听，他听到了一艘海面航船的声音，在以前的航程中这已司空见贯，他不解地看看霍普金斯。但接着听下去，他知道事情不对：与以前不同，这次声音的大小没有变化。'
sample_text = 'During the calm voyage under the sea, Werner fell asleep. I dont know how long it took him to wake up after being pushed by Hopkins. He looked at the chart and track on the small screen and found that two thirds of the voyage had gone, and it seemed that there was nothing unusual. Hopkins asked him to pay attention. He heard the voice of a ship sailing on the sea. He had seen it through the sky during the previous voyage. He looked at Hopkins in puzzlement. But after listening, he knew that something was wrong: different from before, the size of the voice did not change this time'
print(sample_text)
# We'll collect the generated summaries of each model in a dictionary
summaries = {}

During the calm voyage under the sea, Werner fell asleep. I dont know how long it took him to wake up after being pushed by Hopkins. He looked at the chart and track on the small screen and found that two thirds of the voyage had gone, and it seemed that there was nothing unusual. Hopkins asked him to pay attention. He heard the voice of a ship sailing on the sea. He had seen it through the sky during the previous voyage. He looked at Hopkins in puzzlement. But after listening, he knew that something was wrong: different from before, the size of the voice did not change this time


In [10]:
# import nltk
# from nltk.tokenize import sent_tokenize

# nltk.download("punkt")

In [12]:
#hide_output
from transformers import pipeline, set_seed

set_seed(42)
pipe = pipeline("text-generation", model="gpt2-xl")
gpt2_query = sample_text + "\nTL;DR:\n"
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)


Some weights of GPT2Model were not initialized from the model checkpoint at gpt2-xl and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'h.12.attn.masked_bias', 'h.13.attn.masked_bias', 'h.14.attn.masked_bias', 'h.15.attn.masked_bias', 'h.16.attn.masked_bias', 'h.17.attn.masked_bias', 'h.18.attn.masked_bias', 'h.19.attn.masked_bias', 'h.20.attn.masked_bias', 'h.21.attn.masked_bias', 'h.22.attn.masked_bias', 'h.23.attn.masked_bias', 'h.24.attn.masked_bias', 'h.25.attn.masked_bias', 'h.26.attn.masked_bias', 'h.27.attn.masked_bias', 'h.28.attn.masked_bias', 'h.29.attn.masked_bias', 'h.30.attn.masked_bias', 'h.31.attn.masked_bias', 'h.32.attn.masked_bias', 'h.33.attn.masked_bias', 'h.34.attn.masked_bias', 'h.35.attn.masked_bias'

ValueError: Connection error, and we cannot find the requested files in the cached path. Please try again or make sure your Internet connection is on.

In [7]:
print(pipe_out)

[{'generated_text': '在海下平静的航程中，沃纳睡着了。不知过了多长时间，他被霍普金斯推醒，他看看小屏幕上的海图和航迹，发现航程已走了三分之二，似乎没有什么异常。霍普金斯让他注意听，他听到了一艘海面航船的声音，在以前的航程中这已司空见贯，他不解地看看霍普金斯。但接着听下去，他知道事情不对：与以前不同，这次声音的大小没有变化。\nTL;DR:\n'}]


In [8]:
print(pipe_out[-1]["generated_text"][len(gpt2_query):])

In [ ]:
summaries["gpt1"] = "\n".join(
    sent_tokenize(pipe_out[-1]["generated_text"][len(gpt2_query) :]))
# ————————————————
# 版权声明：本文为CSDN博主「致Great」的原创文章，遵循CC 3.0 BY-SA版权协议，转载请附上原文出处链接及本声明。
# 原文链接：https://blog.csdn.net/yanqianglifei/article/details/124053500